In [8]:
from faker import Faker
import csv

def generate_fake_data(num_rows):
    fake = Faker()
    data = []
    for _ in range(num_rows):
        row = [fake.name(), fake.address(), fake.email(), fake.date_of_birth()]
        data.append(row)
    return data

def write_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Name', 'Address', 'Email', 'Date of Birth'])
        csv_writer.writerows(data)

if __name__ == "__main__":
    num_rows = 17000
    fake_data = generate_fake_data(num_rows)
    file_name = "fake_data.csv"
    write_to_csv(fake_data, file_name)
    print(f"{num_rows} rows of fake data generated and saved to {file_name}.")


17000 rows of fake data generated and saved to fake_data.csv.


In [15]:
a = pd.read_csv("fake_data.csv")
a['ID'] = range(1, len(a) + 1)
a.head()

,Name,Address,Email,Date_of_Birth,ID
0,Robert Jensen,Unit 0078 Box 0203\nDPO AE 82184,bryandavis@example.com,2011/8/24,1
1,Melissa Gentry,"85621 Lawson Ranch Apt. 177\nMarkport, VA 10759",uarnold@example.net,1993/5/16,2
2,Brian Williams,"2300 Lin Islands\nNew Jamie, WV 37803",matthewstephens@example.com,1967/11/28,3
3,Jonathan Nguyen,USNS Green\nFPO AA 52496,nicholas71@example.net,1986/6/13,4
4,Patrick Mcgrath,"1419 Smith Roads Suite 681\nEast Kathleen, GU ...",cruzcynthia@example.com,1992/6/22,5


In [32]:
a.info

<bound method DataFrame.info of                   Name                                            Address  \
0        Robert Jensen                   Unit 0078 Box 0203\nDPO AE 82184   
1       Melissa Gentry    85621 Lawson Ranch Apt. 177\nMarkport, VA 10759   
2       Brian Williams              2300 Lin Islands\nNew Jamie, WV 37803   
3      Jonathan Nguyen                           USNS Green\nFPO AA 52496   
4      Patrick Mcgrath  1419 Smith Roads Suite 681\nEast Kathleen, GU ...   
...                ...                                                ...   
16995   Felicia Wilson       496 Melanie Landing\nLake Susanton, WY 21683   
16996     Kevin Nelson  68355 Michael Squares Suite 122\nEast Devin, W...   
16997   Sara Rodriguez  298 Laura Courts Suite 675\nMaldonadoport, VI ...   
16998   Vincent Harvey         442 Amy Court\nEast Anthonyburgh, WY 23102   
16999      Frank Owens  793 Alexander Courts\nSouth Samuelshire, PR 20472   

                             Email Date_of_

In [44]:
import snowflake.connector
import pandas as pd
import numpy as np

def create_database_table(dataframe, primary_key, database_name, table_name): 
    
 
    # Check if primary_key is a column or a list in the dataframe, composite key
    
    if isinstance(primary_key, str):
        if primary_key not in dataframe.columns:
            raise ValueError("Primary key column not found in the dataframe.")
    elif isinstance(primary_key, list):
        if not all(col in dataframe.columns for col in primary_key):
            raise ValueError("One or more primary key columns not found in the dataframe.")
    else:
        raise ValueError("Invalid primary key type. Expected string or list.")

        
   # Check if primary_key column(s) is/are unique
    if isinstance(primary_key, str):
        if not dataframe[primary_key].is_unique:
            raise ValueError("Primary key column is not unique.")
    elif isinstance(primary_key, list):
        if dataframe.duplicated(subset=primary_key).any():
            raise ValueError("Composite primary key values are not unique.")

    
    connection = snowflake.connector.connect(
        user = 'CLEO',
        password = 'Dbqwbxs964!',
        account = 'mmqmpah-qi19636'
        #user = 'brucewil',
        #password = 'Psnolecy2!',
        #account = 'mognruk-jlb64949'
    )
    cur = connection.cursor() 
    
    #check if the database exists, Connect the database or create it
    cur.execute(f"SHOW DATABASES LIKE '{database_name}'")
    exists = cur.fetchone()

    if exists:
        # Connect it
        cur.execute(f"USE DATABASE {database_name}")
    else:
        # Create it
        cur.execute(f"CREATE DATABASE {database_name}")

        
    # map the data type
    def map_dtype(dtype):
        if dtype == "object":
            return "STRING"
        elif dtype == "int64":
            return "INTEGER"
        elif dtype == "float64":
            return "FLOAT"
        elif dtype == "bool":
            return "BOOLEAN"
        elif dtype == "datetime64[ns]":
            return "TIMESTAMP"
        else:
            return "STRING"  # Default to STRING.

    
    # Generate the SQL CREATE or REPLACE command
    columns = []
    for column, dtype in dataframe.dtypes.items():
        if isinstance(primary_key, str) and column == primary_key:
            columns.append(f"{column} {map_dtype(dtype)} PRIMARY KEY")
        elif isinstance(primary_key, list) and column in primary_key:
            columns.append(f"{column} {map_dtype(dtype)}")
        else:
            columns.append(f"{column} {map_dtype(dtype)}")


    create_table = f"CREATE OR REPLACE TABLE {table_name} (\n"
    create_table += ",\n".join(columns)
    create_table += "\n)"
     
    cur.execute(create_table)

    # Generate the SQL INSERT command
    insert_statement = f"INSERT INTO {table_name} ("
    insert_statement += f", ".join(dataframe.columns) + f") VALUES ("

    # Create a placeholder for each column value in the INSERT statement
    value_placeholder = ', '.join(['%s'] * len(dataframe.columns))

    insert_statement = insert_statement + value_placeholder + ")"

  
    # Create a list to store all the values from the dataframe
        
    all_values = []
    
    # Iterate over each row in the dataframe
    for _, row in dataframe.iterrows():
        values = []
        for value in row.values:
            if pd.isna(value):
                values.append(None)
            elif isinstance(value, str):
                values.append(value)
            elif isinstance(value, pd.Timestamp):
                values.append(f"{value}")
            else:
                values.append(value)
        all_values.append(tuple(values))  # Convert to tuple and append to the list

    # Set batch size
    batch_size = 16000

    # Split the values into batches and insert each batch separately
    for i in range(0, len(all_values), batch_size):
        batch = all_values[i:i + batch_size] #slicing
        
        cur.executemany(insert_statement, batch)

    
    
    #cur.execute(insert_statement, all_values) execute can't run with all values at once
    
    #print(insert_statement, all_values)


#    connection.commit()
#    connection.autocommit(True)
# Define the batch size based on Snowflake's limitation (e.g., 16,000)


In [23]:
a.head()

,Name,Address,Email,Date_of_Birth,ID
0,Robert Jensen,Unit 0078 Box 0203\nDPO AE 82184,bryandavis@example.com,2011/8/24,1
1,Melissa Gentry,"85621 Lawson Ranch Apt. 177\nMarkport, VA 10759",uarnold@example.net,1993/5/16,2
2,Brian Williams,"2300 Lin Islands\nNew Jamie, WV 37803",matthewstephens@example.com,1967/11/28,3
3,Jonathan Nguyen,USNS Green\nFPO AA 52496,nicholas71@example.net,1986/6/13,4
4,Patrick Mcgrath,"1419 Smith Roads Suite 681\nEast Kathleen, GU ...",cruzcynthia@example.com,1992/6/22,5


In [45]:
create_database_table(a, "ID", "Test_Database", "Test_Table")

In [20]:
c = pd.read_csv("Database_Generator_Test.csv")
c.head()

,ID,Variable_1,Variable_2,Variable_3
0,1000,137'''',134,522
1,1001,148,468,144
2,1002,636,393,132
3,1003,453,999,179
4,1004,947,316,925


In [46]:
create_database_table(c, "ID", "Test_Database22", "Test_Table22")

In [ ]:
d = pd.read_csv()
d.head()